Hola **Josue**!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>


# Sprint 10 - Proyecto

## Descripción del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

### Descarga y preparación de datos

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import resample

df=pd.read_csv('/datasets/Churn.csv')

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [4]:
df['Tenure'] = df['Tenure'].fillna(0) #Rellenar con 0 los valores ausentes

In [5]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
df.duplicated().sum()

0

No hay Valores duplicados, pero si habia valores ausentes en la columna de Tenure, es porque probablemente no llevan más de un año, por lo tanto es equivalente a 0, lo rellené.
Ahora que los datos están preparados, vamos a examinar el equilibrio de clases

## Examinar el equilibrio de clases

### 1.1 Cargar Datos

In [7]:
train,valid =train_test_split(df, test_size=0.25, random_state=12345)

features_train = train.drop('Exited',axis=1)
target_train =train['Exited']
features_valid = valid.drop('Exited',axis=1)
target_valid = valid['Exited']

print(features_train.shape)
print(features_valid.shape)

(7500, 13)
(2500, 13)


In [8]:
# Eliminar columnas que no aportan, columnas irrelevantes
drop_cols = ['RowNumber', 'CustomerId', 'Surname']
features_train = features_train.drop(columns=drop_cols)
features_valid = features_valid.drop(columns=drop_cols)

Justificación


RowNumber: solo es un índice, no tiene relación real con que un cliente se vaya.

CustomerId: es un identificador único; el modelo podría “memorizar” clientes en lugar de generalizar.

Surname: es un nombre, pero en este dataset no hay relación real entre apellido y abandono del cliente; si lo codificamos con one-hot podrías generar muchas columnas inútiles.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Bien, ahora tienes los datos listos para el entrenamiento y evaluación de manera adecuada
</div>

### 1.2 Codificación One-Hot

In [9]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)

# Asegurar que ambas tengan las mismas columnas
features_valid = features_valid.reindex(columns=features_train.columns, fill_value=0)

## Modelo Base Random Forest

In [10]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

# Predicciones
preds_valid = model.predict(features_valid)
probs_valid = model.predict_proba(features_valid)[:, 1]

# Métricas
f1 = f1_score(target_valid, preds_valid)
auc = roc_auc_score(target_valid, probs_valid)

print("F1 Score:", f1)
print("AUC-ROC:", auc)

F1 Score: 0.568370986920333
AUC-ROC: 0.8545903783500988


F1 es bajo (0.568) es decir que le cuesta balancear precisión y recall, probablemente por el desbalance de clases

AUC-ROC es alto (0.85) es decir el modelo distingue bien entre clases.

## Mejorar el F1

### Técnica 1 Correción del desequilibrio de clases (class_weight='balanced')

In [11]:
model_balanced = RandomForestClassifier(
    random_state=12345,
    class_weight='balanced'
)
model_balanced.fit(features_train, target_train)

# Predicciones
preds_valid_balanced = model_balanced.predict(features_valid)
probs_valid_balanced = model_balanced.predict_proba(features_valid)[:, 1]

# Métricas
f1_balanced = f1_score(target_valid, preds_valid_balanced)
auc_balanced = roc_auc_score(target_valid, probs_valid_balanced)

print("F1 Score con class_weight balanced:", f1_balanced)
print("AUC-ROC con class_weight balanced:", auc_balanced)

F1 Score con class_weight balanced: 0.5663082437275987
AUC-ROC con class_weight balanced: 0.8552871513162588


class_weight='balanced' no mejoró el F1 (de hecho bajó un poquito) aunque el AUC-ROC se mantuvo casi igual.

Esto pasa mucho según la teoria porque el ajuste de pesos ayuda a que el modelo preste más atención a la clase minoritaria, pero si ya estaba funcionando bien en la separación, puede que no suba el F1.

Ahora intentemos con el  segundo enfoque más fuerte

### Técnica 2 Correción del equilbrio de clases (Oversampling Manual)

In [12]:
# Combinar features y target para el train
train_df = pd.concat([features_train, target_train], axis=1)

# Separar clases
train_majority = train_df[train_df.Exited == 0]
train_minority = train_df[train_df.Exited == 1]

# Oversampling: replicar la clase minoritaria hasta igualar la mayoritaria
train_minority_upsampled = resample(
    train_minority,
    replace=True,                # muestreo con reemplazo
    n_samples=len(train_majority),# igualar número de muestras
    random_state=12345
)

# Unir las dos clases ya balanceadas
train_upsampled = pd.concat([train_majority, train_minority_upsampled])

# Separar de nuevo features y target
features_train_up = train_upsampled.drop('Exited', axis=1)
target_train_up = train_upsampled['Exited']

# Entrenar modelo
model_up = RandomForestClassifier(random_state=12345)
model_up.fit(features_train_up, target_train_up)

# Predicciones
preds_valid_up = model_up.predict(features_valid)
probs_valid_up = model_up.predict_proba(features_valid)[:, 1]

# Métricas
f1_up = f1_score(target_valid, preds_valid_up)
auc_up = roc_auc_score(target_valid, probs_valid_up)

print("F1 Score con oversampling manual:", f1_up)
print("AUC-ROC con oversampling manual:", auc_up)

F1 Score con oversampling manual: 0.5936170212765958
AUC-ROC con oversampling manual: 0.8489234500963116


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Perfecto, técnicas cómo sobre-muestreo o sub-muestreo o la modificación de parámetros en algunos modelos para especificar el desbalance ayuda a los modelos a poder generalizar mejor cada caso y obtener un buen desempeño
</div>

Ya superamos el F1 mínimo de 0.59 que pedía el proyecto (0.5936) y mantuvimos un buen AUC-ROC (0.8489).

Ya probamos 3 enfoques:

Modelo Base (F1=0.568)

class_weight='balanced' (F1=0.566)

Oversampling manual (F1=0.594)

El mejor hasta ahora es el RandomForest con oversampling.

## Conclusión

En este proyecto se desarrolló un modelo para predecir la fuga de clientes en Beta Bank. Tras la limpieza de datos y la preparación de variables, se entrenó un modelo base de Random Forest que alcanzó un F1 de 0.568 y un AUC-ROC de 0.854. Después, se evaluaron distintas técnicas para abordar el desbalance de clases (2). El uso de class_weight='balanced' no mejoró el desempeño, pero la aplicación de oversampling manual permitió aumentar el F1 a 0.594, superando el umbral mínimo requerido de 0.59 del proyecto, manteniendo un AUC-ROC de 0.849. Estos resultados indican que el modelo tiene una buena capacidad de discriminación y un  buen equilibrio entre precisión y recall gracias al balanceo de clases.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Te felicito por el trabajo realizado Josue, se nota que conoces las métricas de evaluación y planteas conclusiones muy acertadas en cuanto a los resultados. Es importante siempre en un problema de ML identificar el balance entre las clases de tu variable objetivo para poder escoger la métrica correcta ya que para datasets desbalanceados la métrica de accuracy suele presentar buenos valores pero en realidad las predicciones no son buenas y es por la forma en cómo se calcula esta métrica que puede llevar a interpretaciones erroneas en cambio el f1-score es la ideal para este tipo de casos ya que maneja mejor este tipo de problemas.
    
Saludos!
</div>